In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS Books (
                    BookID INTEGER PRIMARY KEY,
                    Title TEXT,
                    Author TEXT,
                    ISBN TEXT,
                    Status TEXT
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Users (
                    UserID INTEGER PRIMARY KEY,
                    Name TEXT,
                    Email TEXT
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Reservations (
                    ReservationID INTEGER PRIMARY KEY,
                    BookID INTEGER,
                    UserID INTEGER,
                    ReservationDate TEXT,
                    FOREIGN KEY (BookID) REFERENCES Books(BookID),
                    FOREIGN KEY (UserID) REFERENCES Users(UserID)
                )''')

# Function to add a new book to the database
def add_book():
    title = input("Enter the book title: ")
    author = input("Enter the author: ")
    isbn = input("Enter the ISBN: ")
    status = input("Enter the status: ")
    
    cursor.execute("INSERT INTO Books (Title, Author, ISBN, Status) VALUES (?, ?, ?, ?)",
                   (title, author, isbn, status))
    conn.commit()
    print("Book added successfully!")

# Function to find a book's details based on BookID
def find_book_details():
    book_id = input("Enter the BookID: ")
    
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    
    if book:
        print("Book Details:")
        print("BookID:", book[0])
        print("Title:", book[1])
        print("Author:", book[2])
        print("ISBN:", book[3])
        print("Status:", book[4])
        
        cursor.execute("SELECT * FROM Reservations WHERE BookID=?", (book_id,))
        reservation = cursor.fetchone()
        
        if reservation:
            user_id = reservation[2]
            cursor.execute("SELECT * FROM Users WHERE UserID=?", (user_id,))
            user = cursor.fetchone()
            print("Reserved by:")
            print("UserID:", user[0])
            print("Name:", user[1])
            print("Email:", user[2])
        else:
            print("This book is not reserved.")
    else:
        print("Book not found!")

# Function to find a book's reservation status based on BookID, Title, UserID, or ReservationID
def find_reservation_status():
    search_input = input("Enter the search input: ")
    
    if search_input.startswith("LB"):
        cursor.execute("SELECT * FROM Books WHERE BookID=?", (search_input,))
        book = cursor.fetchone()
        if book:
            cursor.execute("SELECT * FROM Reservations WHERE BookID=?", (book[0],))
            reservation = cursor.fetchone()
            if reservation:
                print("Reservation Status: Reserved")
            else:
                print("Reservation Status: Not Reserved")
        else:
            print("Book not found!")
    elif search_input.startswith("LU"):
        cursor.execute("SELECT * FROM Users WHERE UserID=?", (search_input,))
        user = cursor.fetchone()
        if user:
            cursor.execute("SELECT * FROM Reservations WHERE UserID=?", (user[0],))
            reservation = cursor.fetchone()
            if reservation:
                print("Reservation Status: Reserved")
            else:
                print("Reservation Status: Not Reserved")
        else:
            print("User not found!")
    elif search_input.startswith("LR"):
        cursor.execute("SELECT * FROM Reservations WHERE ReservationID=?", (search_input,))
        reservation = cursor.fetchone()
        if reservation:
            print("Reservation Status: Reserved")
        else:
            print("Reservation not found!")
    else:
        cursor.execute("SELECT * FROM Books WHERE Title=?", (search_input,))
        books = cursor.fetchall()
        if books:
            print("Books Found:")
            for book in books:
                print("BookID:", book[0])
                print("Title:", book[1])
                print("Author:", book[2])
                print("ISBN:", book[3])
                print("Status:", book[4])
        else:
            print("No books found with the given title.")

# Function to find all the books in the database
def find_all_books():
    cursor.execute("SELECT * FROM Books")
    books = cursor.fetchall()
    
    if books:
        print("All Books:")
        for book in books:
            print("BookID:", book[0])
            print("Title:", book[1])
            print("Author:", book[2])
            print("ISBN:", book[3])
            print("Status:", book[4])
    else:
        print("No books found in the database.")

# Function to modify/update book details based on BookID
def modify_book_details():
    book_id = input("Enter the BookID: ")
    
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    
    if book:
        print("Current Book Details:")
        print("BookID:", book[0])
        print("Title:", book[1])
        print("Author:", book[2])
        print("ISBN:", book[3])
        print("Status:", book[4])
        
        new_title = input("Enter the new title (leave blank to keep the current value): ")
        new_author = input("Enter the new author (leave blank to keep the current value): ")
        new_isbn = input("Enter the new ISBN (leave blank to keep the current value): ")
        new_status = input("Enter the new status (leave blank to keep the current value): ")
        
        if new_title:
            cursor.execute("UPDATE Books SET Title=? WHERE BookID=?", (new_title, book_id))
        if new_author:
            cursor.execute("UPDATE Books SET Author=? WHERE BookID=?", (new_author, book_id))
        if new_isbn:
            cursor.execute("UPDATE Books SET ISBN=? WHERE BookID=?", (new_isbn, book_id))
        if new_status:
            cursor.execute("UPDATE Books SET Status=? WHERE BookID=?", (new_status, book_id))
        
        conn.commit()
        print("Book details updated successfully!")
    else:
        print("Book not found!")

# Function to delete a book based on its BookID
def delete_book():
    book_id = input("Enter the BookID: ")
    
    cursor.execute("SELECT * FROM Books WHERE BookID=?", (book_id,))
    book = cursor.fetchone()
    
    if book:
        cursor.execute("SELECT * FROM Reservations WHERE BookID=?", (book_id,))
        reservation = cursor.fetchone()
        
        if reservation:
            cursor.execute("DELETE FROM Reservations WHERE BookID=?", (book_id,))
        
        cursor.execute("DELETE FROM Books WHERE BookID=?", (book_id,))
        conn.commit()
        print("Book deleted successfully!")
    else:
        print("Book not found!")

# Main program loop
while True:
    print("\nLibrary Management System")
    print("1. Add a new book")
    print("2. Find a book's detail based on BookID")
    print("3. Find a book's reservation status")
    print("4. Find all the books in the database")
    print("5. Modify/update book details based on BookID")
    print("6. Delete a book based on BookID")
    print("7. Exit")
    
    choice = input("Enter your choice: ")
    
    if choice == "1":
        add_book()
    elif choice == "2":
        find_book_details()
    elif choice == "3":
        find_reservation_status()
    elif choice == "4":
        find_all_books()
    elif choice == "5":
        modify_book_details()
    elif choice == "6":
        delete_book()
    elif choice == "7":
        break
    else:
        print("Invalid choice. Please try again.")

# Close the database connection
conn.close()


Library Management System
1. Add a new book
2. Find a book's detail based on BookID
3. Find a book's reservation status
4. Find all the books in the database
5. Modify/update book details based on BookID
6. Delete a book based on BookID
7. Exit
Book added successfully!

Library Management System
1. Add a new book
2. Find a book's detail based on BookID
3. Find a book's reservation status
4. Find all the books in the database
5. Modify/update book details based on BookID
6. Delete a book based on BookID
7. Exit
Book Details:
BookID: 1
Title: 杨泓彬卖血记
Author: 幻灭
ISBN: 00000
Status: 0
This book is not reserved.

Library Management System
1. Add a new book
2. Find a book's detail based on BookID
3. Find a book's reservation status
4. Find all the books in the database
5. Modify/update book details based on BookID
6. Delete a book based on BookID
7. Exit
No books found with the given title.

Library Management System
1. Add a new book
2. Find a book's detail based on BookID
3. Find a book's 

In [4]:
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
import pyLDAvis.gensim

# Download the required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('gutenberg')

# Load the Alice's Adventures in Wonderland text
alice = gutenberg.sents('carroll-alice.txt')

# Preprocess the text
stop_words = set(stopwords.words('english'))

# Tokenize and remove stopwords
texts = [
    [word.lower() for word in word_tokenize(' '.join(sentence)) if word.lower() not in stop_words]
    for sentence in alice
]

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(texts)

# Create a corpus from the documents
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary)

# Print the topics generated by the model
print("Topics:")
for topic_id, topic in lda_model.show_topics(num_topics=5, num_words=5):
    print(f"Topic {topic_id}: {topic}")

# Visualize the topics
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12552\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12552\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\12552\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Topics:
Topic 0: 0.176*"'" + 0.123*"," + 0.044*"." + 0.030*"said" + 0.022*"alice"
Topic 1: 0.090*"," + 0.084*"." + 0.038*"said" + 0.018*"alice" + 0.016*"!"
Topic 2: 0.186*"'" + 0.088*"," + 0.036*"." + 0.030*"--" + 0.028*"!"
Topic 3: 0.125*"," + 0.033*"." + 0.029*"'" + 0.018*":" + 0.008*"!"
Topic 4: 0.098*"," + 0.034*"." + 0.028*"alice" + 0.021*"'" + 0.020*":"
